In [590]:
import pandas as pd
import re

In [591]:
#Carga de datos CSV 
data = pd.read_csv("L_STMaestroPrd.csv")
data.columns[data.isna().all()]
df_MasterProductos = data.dropna(axis=1, how='all').copy()
df_MasterProductos["descripcion_limpia_producto"] = df_MasterProductos["descrip1"].fillna('') + " " + df_MasterProductos["descrip2"].fillna('')

data_1 = pd.read_csv('Phoenix.csv')
data_1.columns

C:\Users\OPERADOR\AppData\Local\Temp\ipykernel_4232\3370362041.py:2: DtypeWarning: Columns (7,14,17,18,21,28,39,42,46) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("L_STMaestroPrd.csv")


Index(['PUBLICADO/\nNO PUBLICADO', 'U N', 'LINEA', 'EAN', 'COD. DISPRO',
       'COD. ELEA', 'COD. CID LATINA', 'REGISTRO ALFABETA', 'CODIGO KAIROS',
       'PAMI', 'CERTIF NRO', 'descripcion', 'presentacion'],
      dtype='object')

In [592]:
df_Proveedor = data_1 
df_Proveedor['descripcion_limpia_proveedor'] = df_Proveedor['descripcion']
df_Proveedor['descripcion_limpia_proveedor']

0      3TC + AZT ELEA 
1        ABACAVIR ELEA
2               ALCAF 
3               BENJOR
4         BEVAX 100 MG
            ...       
697         ZATIUM MET
698         ZATIUM MET
699         ZATIUM MET
700            ZOPIROL
701         ZOPIROL DM
Name: descripcion_limpia_proveedor, Length: 702, dtype: object

In [593]:
def procesar_descripcion(data, nombre_columna):
    """
    Procesa una columna de un DataFrame aplicando normalización y manejo de combinaciones de texto.
    
    Parámetros:
    - data: DataFrame que contiene la columna a procesar.
    - nombre_columna: Nombre de la columna que se va a procesar.
    
    Retorna:
    - DataFrame con la columna procesada y actualizada.
    """
    # Verificar si la columna existe en el DataFrame
    if nombre_columna not in data.columns:
        raise KeyError(f"La columna '{nombre_columna}' no existe en el DataFrame.")
    
    # Función para normalizar el texto
    def normalizar(texto):
        # Asegurar que el texto sea un string
        texto = str(texto).strip().lower()
        
        # 1. Eliminar espacios redundantes
        texto = re.sub(r'\s+', ' ', texto)

        # 2. Remover caracteres especiales excepto letras, números, espacios y '%'
        texto = re.sub(r'[^a-zA-Z0-9\s%]', '', texto)

        # 3. Separar números de letras y letras de números
        texto = re.sub(r'(\d)([a-zA-Z])', r'\1 \2', texto)
        texto = re.sub(r'([a-zA-Z])(\d)', r'\1 \2', texto)

        # 4. Manejo de 'x':
        texto = re.sub(r'(?<=\d)x(?=\d)', r' x ', texto)  # separar x entre números
        texto = re.sub(r'\b([a-zA-Z]+)x\b', r'\1 x', texto)  # separar palabras terminadas en x

        # 5. Reemplazar separadores como guiones o barras con espacios
        texto = re.sub(r'[-/]', ' ', texto)

        # 6. Manejo de unidades (por ejemplo, separar g, ml, etc. de los números)
        texto = re.sub(r'(\d+)\s*([a-zA-Z]+)', r'\1 \2', texto)

        # 7. Eliminar espacios redundantes nuevamente
        texto = texto.strip()

        # 8. Combinaciones
        combinaciones = {
            'anemido x': 'anemidox',
            'jgaprell' : 'jga prell',
            'fle x': 'flex',
            'argeflo x':'argeflox',
            'aropa x': 'aropax',
            'arolte x' : 'aroltex',
            'atopi x':  'atopix',
            'atorma x': 'atormax',
            'comprec': 'comp recubierto',
            'soloft': 'solucion oftalmica',
            'g': 'gr',
            'cr': 'crema',
            'comp' : 'comprimidos',
            'caps': 'capsulas',
            "jer.pre" : "jeringa prell",
            "mg" : "miligramos",
            "ml" : "mililitros",
            "mm" : "milimetros",
            'zyvo x': 'zyvox',
            'loc' : 'locion',
            "gr" : "gramos",
            "jga" : 'jeringa',
            "cm" : 'centimetros',
            'u' : "unidad",
            'shamp' : 'shampoo',
            "unid" : " unidad",
            'cep' : 'cepillo dental',
            "emuls prot" : 'emulsion protectora',
            'emulshidrat':'emulsion hidratante',
            'emulshumect' : "emulsion humectante",
            'iny': 'inyeccion',
            'amp' : 'ampolla',
            'inylioffa': 'inyeccion lioffa',
            'ds' : 'dosis',
            'inyfa': 'inyeccion fa',
            'jbe': 'jarabe',
            'blist': 'blister',
            'sob' : 'sobres',
            'fa': 'frasco ampolla',
            'jab' : 'jabon',
            'cps' :'capsulas',
            'ivi x' : 'ivix',
            'inya': 'inyectable',
            'inyjgaprell' : 'iny jeringa prellenada',
            'mcg': 'microgramo',
            'gts': 'gotas',
            'mgml' : 'miligramos por mililitro',
            'env': 'envase',
            'gtsoft' : 'gotas oftálmicas',
            'compdisp': 'comprimidos disp',
            'comprecran' : 'comprimidos recubiertos ranurados',
            'sol': 'solucion',
            'jgapre': 'jeringa prellenada',
            'fcocps' : 'frasco capsulas',
            'fcogotero' : 'frasco gotero',
            'fco': 'frasco', 
            'solspray' : 'solucion spray',
            'solpuas' : 'sol puas',
            'cpsblandas' : 'cps blandas',
            'tab' : 'tableta',
            'sach' : 'sachet',
            'mgvial': 'miligramos vial',
            'cpsbl' :'capsula blanda'
        }
        for key, val in combinaciones.items():
            texto = re.sub(rf'\b{key}\b', val, texto)

        return texto

    # Aplicar la función 'normalizar' a cada elemento de la columna
    data[nombre_columna] = data[nombre_columna].apply(normalizar)
    
    return data  # Retorna el DataFrame con la columna procesada


In [594]:
# Procesamiento de df_MasterProductos
df_MasterProductos = procesar_descripcion(df_MasterProductos, "descripcion_limpia_producto")


# Procesamiento de df_Proveedor
df_Proveedor = procesar_descripcion(data_1, 'descripcion_limpia_proveedor')
df_Proveedor = df_Proveedor.dropna()  # Eliminar filas con NaN

# Ahora ambos DataFrames no tienen filas con NaN


In [595]:
df_Proveedor['descripcion_limpia_proveedor']

76       8 horas 2 miligramos
77       8 horas 3 miligramos
78     abara x 100 miligramos
95                   aeroxina
96                   aeroxina
                ...          
689         zarator plus 2010
690                  zarontin
691                  zarontin
700                   zopirol
701                zopirol dm
Name: descripcion_limpia_proveedor, Length: 343, dtype: object

In [596]:
# Función para filtrar productos por laboratorios sobre la MASTER PRODUCTOS 
def laboratorio_input_df(df, columna):
    input_name = input("Ingrese el nombre del laboratorio: ").strip()
    tablas_coincidencias = df[df[columna].str.contains(input_name, case=False, na=False)].copy()
    if not tablas_coincidencias.empty:
        print(f"Se encontraron {len(tablas_coincidencias)} productos coincidentes del laboratorio '{input_name}'.")
        return tablas_coincidencias
    else:
        print(f"No se encontraron productos del laboratorio '{input_name}'.")
        return None


In [597]:
import pandas as pd

# Función para filtrar los productos según el laboratorio ingresado (sin importar mayúsculas/minúsculas)
def filtrar_por_laboratorio(df, laboratorio_input, columna_laboratorio='atrib0'):
    # Convertir tanto la columna de laboratorios como el input a minúsculas para hacer la comparación insensible a mayúsculas
    df_filtrado = df[df[columna_laboratorio].str.lower() == laboratorio_input.lower()]
    return df_filtrado

# Pedir al usuario que ingrese el nombre del laboratorio
laboratorio_input = input("Ingresa el nombre del laboratorio: ")  # El usuario ingresa el nombre del laboratorio

# Filtrar los productos correspondientes a ese laboratorio
df_MasterProductos_Filtro = filtrar_por_laboratorio(df_MasterProductos, laboratorio_input, columna_laboratorio='atrib0')

# Ver el resultado del dataframe filtrado
df_MasterProductos_Filtro


,niprod,cod_prod,cod_abrev,descrip1,descrip2,abrev,cod_catprod,codbarra,cod_fbarra,unid_v,...,tipo_egreso,pc_diftrans,cod_cvta,cat_item,cant_rep,pto_rep,atrib0,atrib1,atrib2,descripcion_limpia_producto
604,26458.0,33228,33228,ACOTRAL 10 mg comp.x 10,NaN,ACOTRAL 10 mg comp.x,PT,7791848286017,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,PHOENIX,NaN,NaN,acotral 10 miligramos comprimidos x 10
605,26459.0,33229,33229,ACOTRAL 10 mg comp.x 20,NaN,ACOTRAL 10 mg comp.x,PT,7791848286024,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,PHOENIX,NaN,NaN,acotral 10 miligramos comprimidos x 20
606,26460.0,33751,33751,ACOTRAL 10 mg comp.x 30,NaN,ACOTRAL 10 mg comp.x,PT,7791848286031,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,PHOENIX,NaN,NaN,acotral 10 miligramos comprimidos x 30
1266,26461.0,33082,33082,AGIOFIBRAS gran.x 250 g,NaN,AGIOFIBRAS gran.x 25,PT,7791848259066,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,PHOENIX,NaN,NaN,agiofibras gran x 250 gramos
1267,26462.0,46479,46479,AGIOFIBRAS sob.x 12,NaN,AGIOFIBRAS sob.x 12,PT,7791848259097,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,PHOENIX,NaN,NaN,agiofibras sobres x 12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56037,91314.0,23342,23342,XANOL 12 mcg caps.x 30,NaN,XANOL 12 mcg caps.x,PT,7791848262028.0,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,PHOENIX,NaN,NaN,xanol 12 microgramo capsulas x 30
56038,91315.0,25874,25874,XANOL 12 mcg caps.x 60,NaN,XANOL 12 mcg caps.x,PT,7791848262042.0,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,PHOENIX,NaN,NaN,xanol 12 microgramo capsulas x 60
56039,91316.0,23341,23341,XANOL 6 mcg caps.x 30,NaN,XANOL 6 mcg caps.x 3,PT,7791848262011.0,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,PHOENIX,NaN,NaN,xanol 6 microgramo capsulas x 30
56040,91317.0,25873,25873,XANOL 6 mcg caps.x 60,NaN,XANOL 6 mcg caps.x 6,PT,7791848262035.0,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,PHOENIX,NaN,NaN,xanol 6 microgramo capsulas x 60


In [598]:
df_MasterProductos_Filtro[['atrib0', 'descripcion_limpia_producto']]

,atrib0,descripcion_limpia_producto
604,PHOENIX,acotral 10 miligramos comprimidos x 10
605,PHOENIX,acotral 10 miligramos comprimidos x 20
606,PHOENIX,acotral 10 miligramos comprimidos x 30
1266,PHOENIX,agiofibras gran x 250 gramos
1267,PHOENIX,agiofibras sobres x 12
...,...,...
56037,PHOENIX,xanol 12 microgramo capsulas x 30
56038,PHOENIX,xanol 12 microgramo capsulas x 60
56039,PHOENIX,xanol 6 microgramo capsulas x 30
56040,PHOENIX,xanol 6 microgramo capsulas x 60


Función de comparación con similitud de coseno


In [599]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

# Vectorizar todas las descripciones (tanto de la master como del proveedor)
descripciones_completas = df_MasterProductos_Filtro['descripcion_limpia_producto'].tolist() + df_Proveedor['descripcion_limpia_proveedor'].tolist()
vectorizer = TfidfVectorizer().fit_transform(descripciones_completas)

# Calcular similitud de coseno entre las descripciones de la master y del proveedor
cos_sim_matrix = cosine_similarity(vectorizer[:len(df_MasterProductos_Filtro)], vectorizer[len(df_MasterProductos_Filtro):])

# Para cada producto del proveedor, obtener la similitud más alta con los productos de la master
matches = []

# Asegurarse de que el índice de las filas del proveedor no exceda el número de productos en df_Proveedor
for i, prov_row in df_Proveedor.iterrows():
    prov_info = prov_row.to_dict()  # Información completa del producto del proveedor
    
    # Obtener las similitudes entre el producto del proveedor y todos los productos de la master
    if i < cos_sim_matrix.shape[1]:  # Asegurarse de que i no exceda el número de columnas
        similitudes = cos_sim_matrix[:, i]  # Similitudes entre el proveedor i y todos los productos de la master
        
        # Encontrar la mejor coincidencia (la mayor similitud)
        max_similitud = np.max(similitudes)  # La mayor similitud encontrada
        best_match_index = np.argmax(similitudes)  # Índice del producto de la master con la mayor similitud
        
        # Recuperar la fila de la master correspondiente a la mejor coincidencia
        best_match_info = df_MasterProductos_Filtro.iloc[best_match_index].to_dict()
        
        # Unir la información del proveedor y la mejor coincidencia
        best_match_info.update(prov_info)
        best_match_info['similaridad'] = max_similitud  # Añadir la similitud más alta
        
        matches.append(best_match_info)  # Añadir la coincidencia a la lista

# Convertir la lista de matches a un DataFrame
df_matches = pd.DataFrame(matches)

# Si deseas mantener todas las filas del proveedor, incluyendo aquellas sin coincidencias, puedes hacer un merge
df_matches_all = pd.merge(df_Proveedor, df_matches, how='left', on='descripcion_limpia_proveedor')

# Reordenar columnas para que 'similaridad' esté al principio
cols = ['similaridad'] + [col for col in df_matches_all.columns if col != 'similaridad']
df_matches_all = df_matches_all[cols]

# Mostrar el resultado
df_matches_all


,similaridad,PUBLICADO/\nNO PUBLICADO_x,U N_x,LINEA_x,EAN_x,COD. DISPRO_x,COD. ELEA_x,COD. CID LATINA_x,REGISTRO ALFABETA_x,CODIGO KAIROS_x,...,EAN_y,COD. DISPRO_y,COD. ELEA_y,COD. CID LATINA_y,REGISTRO ALFABETA_y,CODIGO KAIROS_y,PAMI_y,CERTIF NRO_y,descripcion_y,presentacion_y
0,0.532125,PUBLICADO,P.C.,Neurociencias,7796285053147,12459,25314,2001603.0,40561,189093.0,...,7.796285e+12,12459.0,25314.0,2001603.0,40561,189093.0,56240020,53652,8 HORAS 2 MG.,Comprimidos x 30
1,0.532125,PUBLICADO,P.C.,Neurociencias,7796285053154,12460,25315,2001604.0,40562,189094.0,...,7.796285e+12,12460.0,25315.0,2001604.0,40562,189094.0,56241310,53652,8 HORAS 3 MG.,Comprimidos x 30
2,0.532125,PUBLICADO,P.C.,Enf. Huérfanas,7796285272616,17145,27261,3853102.0,47976,228262.0,...,7.796285e+12,17145.0,27261.0,3853102.0,47976,228262.0,61497110,56605,ABARAX 100 MG.,Comprimidos x 100
3,0.000000,PUBLICADO,P.C.,Antiinfectivos,7791848250155,18667,25015,2035102.0,30307,137442.0,...,7.791848e+12,18667.0,25015.0,2035102.0,30307,137442.0,50083410,45814,AEROXINA,250 MG. SUSP. FRASCO x 60 ml.
4,0.000000,PUBLICADO,P.C.,Antiinfectivos,7791848250155,18667,25015,2035102.0,30307,137442.0,...,7.791848e+12,18668.0,25002.0,2035103.0,28045,128281.0,42081010,45814,AEROXINA,500 MG. COMP. x 16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,NaN,PUBLICADO,P.C.,Cardio,7796285272685,14665,27268,3846102.0,47817,227222.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
499,NaN,PUBLICADO,P.C.,Neurociencias,7792690022723,6279,22272,3621101.0,6356,47661.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
500,NaN,PUBLICADO,P.C.,Neurociencias,7792690022716,6280,22271,3621102.0,6357,47662.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
501,NaN,PUBLICADO,P.C.,Visual,7796285255169,13702,25516,3628302.0,36159,165772.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [600]:
df_matches_all[["similaridad", 'niprod', 'descripcion_limpia_proveedor', 'descripcion_limpia_producto', 'atrib0']]

,similaridad,niprod,descripcion_limpia_proveedor,descripcion_limpia_producto,atrib0
0,0.532125,90995.0,8 horas 2 miligramos,dacten 32 miligramos comprimidos x 15,PHOENIX
1,0.532125,90995.0,8 horas 3 miligramos,dacten 32 miligramos comprimidos x 15,PHOENIX
2,0.532125,90995.0,abara x 100 miligramos,dacten 32 miligramos comprimidos x 15,PHOENIX
3,0.000000,26458.0,aeroxina,acotral 10 miligramos comprimidos x 10,PHOENIX
4,0.000000,26458.0,aeroxina,acotral 10 miligramos comprimidos x 10,PHOENIX
...,...,...,...,...,...
498,NaN,NaN,zarator plus 2010,NaN,NaN
499,NaN,NaN,zarontin,NaN,NaN
500,NaN,NaN,zarontin,NaN,NaN
501,NaN,NaN,zopirol,NaN,NaN


In [601]:
df_matches_all['descripcion_limpia_proveedor']

0        8 horas 2 miligramos
1        8 horas 3 miligramos
2      abara x 100 miligramos
3                    aeroxina
4                    aeroxina
                ...          
498         zarator plus 2010
499                  zarontin
500                  zarontin
501                   zopirol
502                zopirol dm
Name: descripcion_limpia_proveedor, Length: 503, dtype: object

In [602]:
# Buscar el valor 102876 en la columna 'niprod'
resultado = df_MasterProductos_Filtro[df_MasterProductos_Filtro['niprod'] == 16787]

# Ver el resultado
resultado['descripcion_limpia_producto'].tolist()


[]

In [603]:
# Buscar 'allopurinol fabra' en la columna 'descripcion_limpia_producto' (ajusta el nombre de la columna si es diferente)
dddd = df_Proveedor[df_Proveedor['descripcion_limpia_proveedor'].str.contains('allopurinol 300 miligramos', case=False, na=False)]

# Mostrar el resultado
dddd


,PUBLICADO/\nNO PUBLICADO,U N,LINEA,EAN,COD. DISPRO,COD. ELEA,COD. CID LATINA,REGISTRO ALFABETA,CODIGO KAIROS,PAMI,CERTIF NRO,descripcion,presentacion,descripcion_limpia_proveedor
